# 머신러닝 데이터 전처리
> 수집한 데이터를 머신러닝 알고리즘을 운용할 수 있는 형태로 가공하는 작업을 지칭합니다.  
결측치, 범주형 데이터, 문자열, 날짜 형식 데이터 등 일반적인 데이터에서 발생 가능한 문제를 해결하는 일반적인 방법론을 지칭 하기도 합니다.  
다만 데이터의 특성 및 사용해야 하는 모델 케이스에 따라 수행해야하는 전처리 과정이 달라 기술적인 어려움이 있습니다.  
이번 시간에는 기본적인 전처리 방법들을 소개하고 샘플데이터에 적용하며 전처리 방법을 익혀 보겠습니다.  

## titanic 데이터 설명
> 타이타닉호에 승선했던 탑승자 정보를 담고 있는 데이터입니다. 데이터분석 필드에서는 교육용 및 입문용 데이터로 오랫동안 사랑받아 온 데이터이며  
탑승자 정보를 바탕으로 이 탑승자가 생존했는지 혹은 사망했는지 분류하는 모델링을 목적으로 제작 된 데이터입니다.  

**columns 정보**
- pclass : 객실 등급  
- survived : 생존 유무
- name : 칭호를 포함한 이름
- sex : 성별
- age : 나이
- sibsp : 형제 혹은 부부의 수
- parch : 부모, 혹은 자녀의 수
- ticket : 탑승권 종류 (각 숫자가 의미하는 바는 데이터를 통해 알 수 있는 것이 없을 수 있습니다.)
- fare : 지불한 운임
- cabin : 객실정보
- embarked : 선착장정보
- boat : 탈출한 보트가 있다면 boat 번호
- body : 사망자의 시신 수습 후 부여한 일련번호
- home : 출신

## 결측 데이터 처리

In [ ]:
# 필요모듈 import
import pandas as pd

In [ ]:
# 타이타닉 데이터 로드
df = pd.read_csv('./data/titanic.csv')
df.head()

In [ ]:
df.info()

In [ ]:
# 결측 데이터 분포 시각화
import seaborn as sns
sns.heatmap(df.isna())

In [ ]:
# 상관관계분석
sns.heatmap(df.corr())

이 중 cabin, home.dest 컬럼 데이터의 경우 결측치 비중이 높고 주변 데이터로부터 빈 데이터를 유추할 수 있는 힌트가 없어 삭제 처리 합니다.

In [ ]:
# 채워넣는 작업을 모두 마친 데이터의 경우 결측비 비율이 50% 이상일 경우 컬럼 삭제 (절대적인 것 아님)
df.drop('cabin', axis=1, inplace=True)
df.head()

In [ ]:
del df['home.dest']
df.head()

In [ ]:
df.pop('ticket')
df.head()

In [ ]:
# 결측치 비율 계산
df.info()

In [ ]:
df['age'].isna().sum() / len(df['age']) * 100

In [ ]:
# 여러가지 방법으로 컬럼 삭제
df['age'].mean()

In [ ]:
# 결측치 처리
df['age'].fillna(df['age'].mean(), inplace=True) # age 컬럼의 결측치를 해당 컬럼의 평균값으로 대체

In [ ]:
df['fare'].fillna(df['fare'].mean(), inplace=True)

In [ ]:
df['embarked'].unique() # 선택 데이터의 고윳값을 확인하는 메소드

In [ ]:
df['embarked'].value_counts() # 고윳값의 갯수를 세어 내림차순으로 정렬

In [ ]:
df['embarked'].fillna('S', inplace=True)

In [ ]:
df.info()

boat, body의 경우 유추하여 값을 채워 넣을 수 있는 데이터는 없지만 데이터의 존재 자체로서 생존 여부를 판단 할 수 있는 데이터이기에 데이터를 명목화 시키겠습니다.

In [ ]:
df.head()

## 범주형 변수의 처리
> 카테고리컬 데이터라고도 하며 데이터가 특정 구간에 속하여 구분이 가능한 데이터 분포를 보이는 변수의 경우 범주형 데이터라 합니다.  
범주형 데이터의 경우 보통 문자열 구분이 되어있어 컴퓨터가 인식 가능한 숫자형태로 바꿔주는데 이를 더미화, 혹은 원핫인코딩이라 합니다.

In [ ]:
# 보트 컬럼의 결측치 여부를 이진형 데이터로 치환
df['boat'] = df['boat'].isna()

In [ ]:
df['body'] = df['body'].isna()

In [ ]:
df.head()

In [ ]:
df.info()

## 카테고리가 여러개인 변수 처리 (문자열 처리)

이름의 경우 미들네임에 따라 생존에 영향을 줄 수 있는 경우가 있습니다. 전체 미들네임에 해당하는 카테고리를 선택하지 않고 일부만을 데이터화 시킵니다.

In [ ]:
df.head()

In [ ]:
# name 컬럼 첫번째 인덱스 데이터 선택해서 test_text로 저장
test_text = df.loc[1, 'name']
test_text

In [ ]:
# 팬시인덱싱 내가 설정한 조건에 따라 데이터 선별
test_text.split()[1]

In [ ]:
# 미들네임 선택 함수
def make_name(x):
    return x.split()[1]

In [ ]:
make_name(test_text)

In [ ]:
# 미들네임 선택 함수 적용 후 name 컬럼 원본값 변경
df['name'] = df['name'].apply(make_name)

In [ ]:
df['name'].unique()

In [ ]:
# name 컬럼의 문자열 그대로는 학습에 사용이 불가능하기에 이를 다시 숫자형태로 치환 + 기타 미들네임을 0으로 변환
def make_name2(x):
    if x == 'Mrs.':
        return 1
    elif x == 'Miss.':
        return 2
    elif x == 'Mr.':
        return 3
    elif x == 'Master.':
        return 4
    else:
        return 0

In [ ]:
# 커스텀함수 적용
df['name'] = df['name'].apply(make_name2)

In [ ]:
df.head()

In [ ]:
# 데이터프레임 내부에 카테고리컬 컬럼을 원핫인코딩 변환 작업
data = pd.get_dummies(df, columns=['pclass', 'name', 'sex', 'embarked', 'boat', 'body'], drop_first=True)

In [ ]:
data.head()

## 타겟, 학습데이터 분할

In [ ]:
# 타겟데이터 분할
y = data['survived']
X = data.drop('survived', axis=1)
# 학습데이터 분할
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

## 모델학습

In [ ]:
# 모델 생성
from sklearn.linear_model import LogisticRegression

In [ ]:
model = LogisticRegression(random_state=42)

In [ ]:
# 모델 학습
model.fit(X_train, y_train)

In [ ]:
# 모델 예측값 저장
pred = model.predict(X_test)

In [ ]:
# 모델 평가
from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(y_test, pred))
print(classification_report(y_test, pred))

## 날짜형식 데이터 전처리

### 사용 데이터 간략 설명
> 한국에너지관리공단에서 제공한 전력사용량 데이터  
1시간 간격으로 수집 된 60개 건물들의 2020년 6월 1일 부터 2020년 8월 24일까지의 데이터  
건물정보와 기후정보를 활용한 전력사용량을 예측하기 위한 데이터셋  
대회홈페이지 : https://dacon.io/competitions/official/235736/overview/description
>> 각 변수(컬럼) 설명  
>>- num : 건물번호  
>>- date_time : 데이터가 수집 된 날짜, 시간  
>>- 전력사용량 : 수집 된 시점에 사용한 전력량  
>>- 비전기냉방설비운영 : 0-미운영, 1-운영  
>>- 태양광보유 : 0-미보유, 1-보유

In [ ]:
# 데이터 로딩
df = pd.read_csv('./data/energy1.csv', encoding='cp949')
df.head()

In [ ]:
# date_time컬럼 날짜 형식으로 변환 후 새로운 변수로 저장
df.info()

In [ ]:
# datetime 형식의 데이터에서 시간 속성 값 추출 테스트
df['date_time'] = pd.to_datetime(df['date_time'])

In [ ]:
# 함수 제작 및 적용
test_time = df.loc[0, 'date_time']

In [ ]:
print(test_time)
print(test_time.year)
print(test_time.month)
print(test_time.day)
print(test_time.hour)
print(test_time.minute)
print(test_time.second)
print(test_time.microsecond)
print(test_time.dayofweek) # 요일 0부터 월요일

In [ ]:
test_time = df.loc[0, 'date_time']
test_time2 = df.loc[1, 'date_time']

In [ ]:
test_time2 - test_time

In [ ]:
# 시간 속성 분해 커스텀 함수 제작
def make_month(x):
    return x.month
def make_day(x):
    return x.day
def make_hour(x):
    return x.hour

In [ ]:
df['month'] = df['date_time'].apply(make_month)
df['day'] = df['date_time'].apply(make_day)
df['hour'] = df['date_time'].apply(make_hour)

In [ ]:
df.head()

In [ ]:
df.loc[행(인덱스, 슬라이싱), 열(문자, 슬라이싱)] # 인덱스 넘버, 문자열을 받아들입니다.

In [ ]:
df.loc[(df['num'] == 1) & (df['month'] == 7), '전력사용량(kWh)'].mean()

In [ ]:
df.loc[df['num'] == 1]

In [ ]:
# 팬시 인덱싱 1번 건물의 7월달 전력사용량 평균
df.loc[df['num'] == 1]